In [1]:
#https://youtu.be/5ypQIUbpA7c?si=OjF6L76MTJJgSrBs
# works 

In [2]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from PIL import Image
import torch
import cv2
from transformers import GLPNImageProcessor, GLPNForDepthEstimation

2024-09-04 19:01:57.443104: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
image_file = "images/pointing.jpg"
video_file = "Videos/pointing.mp4"
video_file = "Videos/pointing.MOV"
image_file = "images/tmp2.jpg"

In [4]:
def extract_img(video_file):
    cam = cv2.VideoCapture(video_file)
    for i in range(0,40):
        ret,frame = cam.read() # read 1st image
    cv2.imwrite("./images/tmp2.jpg", frame)
    cam.release()
    return

extract_img(video_file)

In [5]:
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

In [6]:
# detection
from ultralytics import YOLOWorld

def obj_detect(image):
    # Initialize a YOLO-World model
    model = YOLOWorld("yolov8s-world.pt")  # or select yolov8m/l-world.pt for different sizes
    model.set_classes(["laptop" , "lamp", "fan"])
    results = model.predict(image)

    for result in results:
        print("box-> ")
        print(result.boxes)
        print(result.probs)

    # Show results
    results[0].show()
    return results[0]
bounding=obj_detect(image_file)


image 1/1 /Users/davidtung/Documents/GitHub/GWU-pointing/notebooks/images/tmp2.jpg: 384x640 1 lamp, 164.9ms
Speed: 2.6ms preprocess, 164.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
box-> 
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.])
conf: tensor([0.8806])
data: tensor([[290.1316, 201.0190, 399.8622, 393.7111,   0.8806,   1.0000]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([1, 6])
xywh: tensor([[344.9969, 297.3651, 109.7307, 192.6921]])
xywhn: tensor([[0.2695, 0.4130, 0.0857, 0.2676]])
xyxy: tensor([[290.1316, 201.0190, 399.8622, 393.7111]])
xyxyn: tensor([[0.2267, 0.2792, 0.3124, 0.5468]])
None


In [7]:
image = Image.open(image_file)#.convert("RGB")
new_height = 480 if image.height > 480 else image.height
new_height -= (new_height %32)
new_width = int(new_height * image.width / image.height)
diff = new_height % 32

new_width = new_width - diff if diff < 16 else new_width +32 - diff
new_size = (new_width,new_height)
image = image.resize (new_size)

In [8]:
inputs = feature_extractor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model (**inputs)
    predicted_depth = outputs.predicted_depth
    

In [9]:
pad = 16
#pad = (image.width - predicted_depth.shape[0]) // 2  # p for padding
print(pad)
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad ))

16


In [10]:
import numpy as np

def depth_to_point_cloud(depth_map, scale_factor=0.001):
    h, w = depth_map.shape
    i, j = np.meshgrid(np.arange(w), np.arange(h), indexing='xy')
    z = depth_map * scale_factor
    x = (i - w / 2) * z / w
    y = (j - h / 2) * z / h
    z = -z  # Invert the y-axis
    y = -y  # Invert the y-axis
    points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
    return points

point_cloud = depth_to_point_cloud(output)
print(point_cloud)


[[   -0.97031     0.97031     -1.9406]
 [   -0.97027      0.9727     -1.9454]
 [   -0.97089     0.97577     -1.9515]
 ...
 [    0.90149    -0.90424     -1.8166]
 [    0.90326    -0.90375     -1.8156]
 [    0.90616    -0.90437     -1.8169]]


In [11]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_point_cloud(points):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=0.1)  # Plot the points
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()


In [12]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(image)
#ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap='plasma')

plt.tight_layout()
plt.pause(5)

In [13]:
#

In [13]:
import numpy as np
import open3d as o3d

In [14]:
# Define transformation matrices
rotation_y_180 = np.array([
    [-1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])

rotation_z_180 = np.array([
    [-1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

In [15]:
def apply_transformation2(points, transformation_matrix):
    ones = np.ones((points.shape[0], 1))
    points_homogeneous = np.hstack((points, ones))  # Convert to homogeneous coordinates
    transformed_points_homogeneous = points_homogeneous @ transformation_matrix.T
    transformed_points = transformed_points_homogeneous[:, :3]  # Convert back to 3D coordinates
    return transformed_points

In [16]:
def apply_transformation(point_cloud, transformation_matrix):
    point_cloud.transform(transformation_matrix)
    return point_cloud

In [17]:
def extract_3d_coordinates(point_cloud):
    points = np.asarray(point_cloud.points)
    return points

In [18]:
width, height = image.size
depth_image = (output * 255 / np.max(output)).astype('uint8')
image = np.array(image)

In [19]:
def show_o3d_image(o3d_image):
    np_image = np.asarray(o3d_image)

    # Display the image using Matplotlib
    plt.imshow(np_image)
    plt.axis('off')  # Turn off axis labels
    plt.show()

In [20]:

color_height, color_width = image.shape[:2]
depth_image = cv2.resize(depth_image, (color_width, color_height), interpolation=cv2.INTER_NEAREST)

depth_03d = o3d.geometry.Image(depth_image)
image_03d = o3d.geometry.Image(image)
print(depth_image.shape)
print(image.shape)
#show_o3d_image(depth_image)
#show_o3d_image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth \
(image_03d,depth_03d, depth_scale=1000.0, depth_trunc=3.0,\
 convert_rgb_to_intensity=False)

(448, 821)
(448, 821, 3)


In [21]:
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width/2, height/2)


In [22]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)

In [23]:
# Apply transformations
pcd = apply_transformation(pcd, rotation_y_180)
pcd = apply_transformation(pcd, rotation_z_180)


In [ ]:

#print(np.asarray((pcd.points)))
o3d.visualization.draw_geometries([pcd])



In [ ]:
points = extract_3d_coordinates(pcd)
print(points)